# Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.signal import savgol_filter

# Read data

In [ ]:
# Read the CSV file
df = pd.read_csv(
    "/home/m3rc7pi/workspace/arc/log/20240317_free_fall.csv", 
    header=None, 
    names=["timestamp", "acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"],
    skiprows=0,
)
df['timestamp'] = df['timestamp'] / 1e6

In [ ]:

df["delta_time"] = df['timestamp'].diff() * 1000
Fs = 1 / (df["delta_time"][2:].mean()/1000)  # Sampling frequency (Hz)

plt.figure(figsize=(12, 3))
plt.plot(df['timestamp'][2:], df["delta_time"][2:])  # Plot only positive frequencies
plt.xlabel('Time (s)')
plt.ylabel('dt [ms]')
plt.title('Spectral Analysis of acc_x')
plt.grid(True)
plt.show()

print(df["delta_time"][2:].describe())
print("\nAverage sampling frequency:", Fs, "[Hz]")

# Accelerometer

In [ ]:
# Compute statistics for acceleration data
df['acc_magnitude'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2)

print("\nAcceleration Statistics:")
print(df[["acc_x", "acc_y", "acc_z", 'acc_magnitude']].describe())

In [ ]:
# Plot accelerometer data
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['acc_x'], label='acc_x')
axs[0].plot(df['timestamp'], df['acc_y'], label='acc_y')
axs[0].plot(df['timestamp'], df['acc_z'], label='acc_z')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Accelerometer Data')
axs[0].grid(True)
axs[0].legend()

axs[1].plot(df['timestamp'], df['acc_magnitude'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Acceleration [g]')
axs[1].set_title('Magnitude')
axs[1].grid(True)
axs[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
df['acc_x_filtered'] = savgol_filter(df['acc_x'], window_length=51, polyorder=3)

# Compute the difference between acc_x and acc_x_filtered
df['acc_x_noise'] = df['acc_x'] - df['acc_x_filtered']

# Perform FFT on acc_x
acc_x_fft = np.fft.fft(df['acc_x'])
freq = np.fft.fftfreq(len(df['acc_x']), d=1/Fs)  # Frequency bins

# Perform FFT on acc_x_filtered
acc_x_filtered_fft = np.fft.fft(df['acc_x_filtered'])
freq_filtered = np.fft.fftfreq(len(df['acc_x_filtered']), d=1/Fs)  # Frequency bins

fig, axs = plt.subplots(4, 1, figsize=(12, 12))

axs[0].plot(df['timestamp'], df['acc_x'], label='acc_x')
axs[0].plot(df['timestamp'], df['acc_x_filtered'], label='acc_x filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Filtered X Accelerometer')
axs[0].legend()
axs[0].grid()

axs[1].plot(df['timestamp'], df['acc_x_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[m/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.5, 0.5)
axs[1].legend()
axs[1].grid()

axs[2].plot(freq[0:len(freq)//2], np.abs(acc_x_fft[0:len(freq)//2]))  # Plot only positive frequencies
axs[2].set_xlabel('Frequency (Hz)')
axs[2].set_ylabel('Amplitude')
axs[2].set_title('Spectral Analysis of acc_x')
axs[2].grid()

axs[3].plot(freq_filtered[0:len(freq_filtered)//2], np.abs(acc_x_filtered_fft[0:len(freq_filtered)//2]))
axs[3].set_xlabel('Frequency (Hz)')
axs[3].set_ylabel('Amplitude')
axs[3].set_title('Spectral Analysis of acc_x_filtered')
axs[3].grid()

plt.tight_layout()
plt.show()

print(df[["acc_x", "acc_x_filtered", "acc_x_noise"]].describe())

In [ ]:
df['acc_y_filtered'] = savgol_filter(df['acc_y'], window_length=51, polyorder=3)

# Compute the difference between acc_y and acc_y_filtered
df['acc_y_noise'] = df['acc_y'] - df['acc_y_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['acc_y'], label='acc_y')
axs[0].plot(df['timestamp'], df['acc_y_filtered'], label='acc_y filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Filtered Y Accelerometer')
axs[0].legend()
axs[0].grid()

axs[1].plot(df['timestamp'], df['acc_y_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[m/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.5, 0.5)
axs[1].legend()
axs[1].grid()

plt.tight_layout()
plt.show()

print(df[["acc_y", "acc_y_filtered", "acc_y_noise"]].describe())

In [ ]:
df['acc_z_filtered'] = savgol_filter(df['acc_z'], window_length=51, polyorder=3)

# Compute the difference between acc_z and acc_z_filtered
df['acc_z_noise'] = df['acc_z'] - df['acc_z_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['acc_z'], label='acc_z')
axs[0].plot(df['timestamp'], df['acc_z_filtered'], label='acc_z filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Acceleration [m/s]')
axs[0].set_title('Filtered Z Accelerometer')
axs[0].legend()
axs[0].grid()

axs[1].plot(df['timestamp'], df['acc_z_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[m/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.5, 0.5)
axs[1].legend()
axs[1].grid()

plt.tight_layout()
plt.show()

print(df[["acc_z", "acc_z_filtered", "acc_z_noise"]].describe())

# Gyroscope

In [ ]:
# Compute statistics for gyro data
gyro_stats = df[["gyro_x", "gyro_y", "gyro_z"]].describe()
print("\nGyro Statistics:")
print(gyro_stats)

In [ ]:
# Plot gyroscope data
plt.figure(figsize=(12, 3))
plt.plot(df['timestamp'], df['gyro_x'], label='gyro_x')
plt.plot(df['timestamp'], df['gyro_y'], label='gyro_y')
plt.plot(df['timestamp'], df['gyro_z'], label='gyro_z')
plt.xlabel('Timestamp')
plt.ylabel('Angular Velocity')
plt.title('Gyroscope Data')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
df['gyro_x_filtered'] = savgol_filter(df['gyro_x'], window_length=51, polyorder=3)

# Compute the difference between gyro_x and gyro_x_filtered
df['gyro_x_noise'] = df['gyro_x'] - df['gyro_x_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['gyro_x'], label='gyro_x')
axs[0].plot(df['timestamp'], df['gyro_x_filtered'], label='gyro_x filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Angular Velocity [rad/s]')
axs[0].set_title('Filtered X Angular Velocity')
axs[0].grid(True)
axs[0].legend()

axs[1].plot(df['timestamp'], df['gyro_x_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[rad/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.05, 0.05)
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()

print(df[["gyro_x", "gyro_x_filtered", "gyro_x_noise"]].describe())

In [ ]:
df['gyro_y_filtered'] = savgol_filter(df['gyro_y'], window_length=51, polyorder=3)

# Compute the difference between gyro_y and gyro_y_filtered
df['gyro_y_noise'] = df['gyro_y'] - df['gyro_y_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['gyro_y'], label='gyro_y')
axs[0].plot(df['timestamp'], df['gyro_y_filtered'], label='gyro_y filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Angular Velocity [rad/s]')
axs[0].set_title('Filtered Y Angular Velocity')
axs[0].legend()
axs[0].grid(True)

axs[1].plot(df['timestamp'], df['gyro_y_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[rad/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.05, 0.05)
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

print(df[["gyro_y", "gyro_y_filtered", "gyro_y_noise"]].describe())

In [ ]:
df['gyro_z_filtered'] = savgol_filter(df['gyro_z'], window_length=51, polyorder=3)

# Compute the difference between gyro_z and gyro_z_filtered
df['gyro_z_noise'] = df['gyro_z'] - df['gyro_z_filtered']

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
axs[0].plot(df['timestamp'], df['gyro_z'], label='gyro_z')
axs[0].plot(df['timestamp'], df['gyro_z_filtered'], label='gyro_z filtered')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Angular Velocity [rad/s]')
axs[0].set_title('Filtered Y Angular Velocity')
axs[0].legend()
axs[0].grid(True)

axs[1].plot(df['timestamp'], df['gyro_z_noise'], label='diff')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('[rad/s]')
axs[1].set_title('Noise')
axs[1].set_ylim(-0.05, 0.05)
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

print(df[["gyro_z", "gyro_z_filtered", "gyro_z_noise"]].describe())